In [5]:
import os
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [6]:
folder = "/Users/annek/Library/CloudStorage/OneDrive-SharedLibraries-NIOZ/PhD Anneke Vries - General/Data/Freshwater/"
# make a list of all files in the folder
files = os.listdir(folder)
# sort files alphabetically
print(files)

['KangiataNunaataSermia_SW_D275.csv', 'NuupKangerdlua_solid_ice_discharge_Gt.csv', 'NarsapSermia_SW_D262.csv', 'AkullersuupSermia_SW_D270.csv']


In [7]:
dfKNS = pd.read_csv(folder + 'KangiataNunaataSermia_SW_D275.csv', index_col=0, parse_dates=True, header=2)
dfKNS = dfKNS/1e9 # convert to Gt
# read first line of file 
dfKNS_info = pd.read_csv(folder + 'KangiataNunaataSermia_SW_D275.csv', header=None)[:3]

fig = px.area(dfKNS.loc[:, :"BasalMelt"], title='Kangiata Nunaata Sermia', template='simple_white')
# fig.add_trace(px.line(dfKNS.loc[:, "Total_MassLoss"], title='Kangiata Nunaata Sermia').data[0])
# set ylabel
fig.update_yaxes(title_text='Mass loss (Gt)')
fig.update_xaxes(title_text="")
# can you add ticks for every year?
fig.update_xaxes(tickformat="%Y", ticklabelmode="period", dtick="M12")


In [8]:
dfNS = pd.read_csv(folder + 'NarsapSermia_SW_D262.csv', index_col=0, parse_dates=True, header=2)
dfNS = dfNS/1e9 # convert to Gt
dfNS_info = pd.read_csv(folder + 'NarsapSermia_SW_D262.csv', header=None)[:3]

fig2 = px.area(dfNS.loc[:, :"BasalMelt"], title='Narsap Sermia', template='simple_white')
# fig2.add_trace(px.line(dfNS.loc[:, "Total_MassLoss"], title='Narsap Sermia').data[0])

fig2.update_yaxes(title_text='Mass loss (Gt)')
fig2.update_xaxes(title_text="")
fig2.update_xaxes(tickformat="%Y", ticklabelmode="period", dtick="M12")

In [9]:
dfAS = pd.read_csv(folder + 'AkullersuupSermia_SW_D270.csv', index_col=0, parse_dates=True, header=2)
dfAS = dfAS/1e9 # convert to Gt\
dfAS_info = pd.read_csv(folder + 'AkullersuupSermia_SW_D270.csv', header=None)[:3]

fig3 = px.area(dfAS.loc[:, :"BasalMelt"], title='Akullersuup Sermia', template='simple_white')
# fig3.add_trace(px.line(dfAS.loc[:, "Total_MassLoss"], title='Akullersuup Sermia').data[0])

fig3.update_yaxes(title_text='Mass loss (Gt)')
fig3.update_xaxes(title_text="")
fig3.update_xaxes(tickformat="%Y", ticklabelmode="period", dtick="M12")


In [10]:


fig = make_subplots(rows=1, cols=1, shared_yaxes=True)

fig.add_trace(go.Scatter(x=dfAS.index, y=dfAS["Total_MassLoss"],  name='Akullersuup Sermia'), row=1, col=1, )
fig.add_trace(go.Scatter(x=dfKNS.index, y=dfKNS["Total_MassLoss"], name='Kangiata Nunaata Sermia'), row=1, col=1)
fig.add_trace(go.Scatter(x=dfNS.index, y=dfNS["Total_MassLoss"], name='Narsap Sermia'), row=1, col=1)

fig.update_yaxes(title_text='Mass loss (Gt)')
fig.update_layout(template='simple_white')
fig.update_xaxes(title_text="", tickformat="%Y", ticklabelmode="period", dtick="M12", row=3, col=1)

fig.show()

In [11]:
# sum all three df together
df_NK = dfKNS + dfNS + dfAS

fig = px.area(df_NK.loc[:, :"BasalMelt"], title='Narsap Kangiata Sermia', template='simple_white')
# fig.add_trace(px.line(df_NK.loc[:, "Total_MassLoss"], title='Narsap Kangiata Sermia').data[0])
fig.show()

In [12]:
# average per year df_NK

df_monthly_mean = df_NK.groupby(df_NK.index.month).mean()
fig = px.area(df_NK.groupby(df_NK.index.month).mean().loc[:, :"BasalMelt"]', template='simple_white')

fig.update_yaxes(title_text='Mass loss (Gt)')
fig.update_xaxes(title_text="")

title = 'Average monthly mass loss between 2010 and 20121'
fig.update_layout(title=title)
fig.show()

In [13]:


# make a cumulative line plot 
df_NKCumsum = df_NK.groupby(df_NK.index.year).cumsum()
df_NKCumsum["month"] = df_NK.index.month
df_NKCumsum["year"] = df_NK.index.year

px.line(df_NKCumsum, x="month", y="SurfaceMelt", color="year", title='Cumulative mass loss between 2010 and 2021', template='simple_white')

/Users/annek/opt/anaconda3/envs/phdIIfresh/lib/python3.12/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [76]:
(df_NK.groupby(df_NK.index.year).sum().mean()/df_NK.groupby(df_NK.index.year).sum().mean()['Total_MassLoss']*100).round(1)

SurfaceMelt        52.9
IceDischarge       43.1
BasalMelt           4.0
Total_MassLoss    100.0
dtype: float64

# Gt to melt energy needed

In [26]:
def Gt_solid_to_melt_energy(Gt_ice):
    '''Convert Gt to energy in J'''
    kg_solid_discharge = Gt_ice *1e12

    T_ice = -10 # degrees C
    heat_capacity_ice = 2100 # J/kg/K 
    EnergyHeatingOfIce = (0 - T_ice) * heat_capacity_ice * kg_solid_discharge #J
    
    heat_fusion_ice = 334000 # J/kg
    EnergyMeltIce =  kg_solid_discharge * heat_fusion_ice #J    
    return (EnergyHeatingOfIce + EnergyMeltIce)   # J


Gt into Nuup Kangerlua over Mooring time scale

In [27]:
df_NK['2018-06':'2019-06'].sum()

SurfaceMelt        9.554199
IceDischarge       8.522674
BasalMelt          0.752551
Total_MassLoss    18.829424
dtype: float64

Converted into Joule needed

In [68]:
Gt_ice = 0.656
kg_solid_discharge = Gt_ice *1e12

T_ice = -10 # degrees C
heat_capacity_ice = 2100 # J/kg/K 
EnergyHeatingOfIce = (0 - T_ice) * heat_capacity_ice * kg_solid_discharge #J

heat_fusion_ice = 334000 # J/kg
EnergyMeltIce =  kg_solid_discharge * heat_fusion_ice #J    

print(f"It takes {EnergyHeatingOfIce/1e12:.1e} TJ to heat {Gt_ice} Gt of ice from -10 to 0 degrees C")
print(f"It takes {EnergyMeltIce/1e12:.1e} TJ to melt {Gt_ice} Gt of ice at 0 degrees C")
print(f"So it takes this much energy to melt {Gt_ice} Gt of ice: \n{(EnergyHeatingOfIce+EnergyMeltIce)/1e15:.1f} PJ")

# number of seconds from 2010-06-01 to 2019-06-30
import datetime
d0 = datetime.datetime(2018, 7, 1)
d1 = datetime.datetime(2018, 7, 31)
delta_seconds = (d1 - d0).total_seconds()

print(f"So much heat transport: {(EnergyHeatingOfIce+EnergyMeltIce)/delta_seconds /1e9:.0f} GW")


It takes 1.4e+04 TJ to heat 0.656 Gt of ice from -10 to 0 degrees C
It takes 2.2e+05 TJ to melt 0.656 Gt of ice at 0 degrees C
So it takes this much energy to melt 0.656 Gt of ice: 
232.9 PJ
So much heat transport: 90 GW


In [70]:
Gt_solid_to_melt_energy(8.52)/1e15


178.92000000284568

In [58]:
df_monthly_mean = df_NK.groupby(df_NK.index.month).mean()
fig = px.area( df_NK['2018-06':'2019-06'].loc[:, :"BasalMelt"],  template='simple_white')

fig.update_yaxes(title_text='Mass loss (Gt)')
fig.update_xaxes(title_text="")

title = 'Average monthly mass loss between 2010 and 20121'
fig.update_layout(title=title)
fig.show()

In [59]:
df_NK['2018-06':'2019-06'].mean()

SurfaceMelt       0.734938
IceDischarge      0.655590
BasalMelt         0.057889
Total_MassLoss    1.448417
dtype: float64